# Feedly の Saved for Later に登録された似た記事をレコメンドしてみる

"この記事を読んでいる人はこれも読んでます" 的な機能をさっくり作りたかった

## 1. Feedly から Saved for Later の一覧を JSON 形式で取得する

[FeedlyのSaved For LaterをエクスポートしてPocketへ移行する方法](http://tsukuba42195.top/?p=927) を参考に json データとして取得

ちょっと改良して、記事の概要部分もあわせて JSON 化するように修正

上記記事中の `saveToFile` 関数内で、`.u0Summary` も一緒に取ってくるように修正

```javascript
 function saveToFile() {
   // Loop through the DOM, grabbing the information from each bookmark
   map = jQuery("#section0_column0 div.u0Entry").map(function(i, el) {
     var eel = jQuery(el);
     var regex = /published:(.*)\ --/i;
     var abstract = jQuery(".u0Summary", el).text(); // <---- 概要部分も取得
     return {
       title: eel.data("title"),
       abstract: abstract,
       url: eel.data("alternate-link"),
       time: regex.exec(eel.find("div.lastModified span").attr("title"))[1],
     };
   }).get(); //  Convert jQuery object into an array
   // Convert to a nicely indented JSON string
   json = JSON.stringify(map, undefined, 2);
 var blob = new Blob([json], {type: "text/plain;charset=utf-8"});
 saveAs(blob, "FeedlySavedForLater" + Date.now().toString() + ".txt");
 }
```

### 実際の取得例

```json
[
  {
    "title": "Qiitaレジェンド達の偉大さをシェル芸で眺めて2016年を振り返る",
    "url": "http://qiita.com/t_nakayama0714/items/776724410b2a119af088",
    "time": "Wed, 28 Dec 2016 14:46:40 GMT"
  },
  {
    "title": "シャイハックの心得を説く：『消極性デザイン宣言』出版記念イベントレポート",
    "url": "https://codeiq.jp/magazine/2016/12/48292/",
    "time": "Wed, 28 Dec 2016 03:00:42 GMT"
  },
  {
    "title": "2016年はてなWebオペレーションエンジニアのアウトプット",
    "url": "http://developer.hatenastaff.com/entry/2016/12/28/151403",
    "time": "Wed, 28 Dec 2016 06:14:03 GMT"
  },
  {
    "title": "メカデザイナー 大久保淳二インタビュー 架空企業が描く未来都市の構想",
    "url": "http://kai-you.net/article/36575",
    "time": "Thu, 22 Dec 2016 09:00:00 GMT"
  },
  {
    "title": "ReduxのAction、Reducer、Storeの（個人的な）整理メモ",
    "url": "http://qiita.com/kmszk/items/4481e9dec0249cb34fc9",
    "time": "Fri, 23 Dec 2016 06:12:40 GMT"
  },
  ...
```

※ 掲載/利用記事
- http://qiita.com/t_nakayama0714/items/776724410b2a119af088
- https://codeiq.jp/magazine/2016/12/48292/
- http://developer.hatenastaff.com/entry/2016/12/28/151403
- http://kai-you.net/article/36575
- http://qiita.com/kmszk/items/4481e9dec0249cb34fc9

### SavedForLater に登録されている記事件数

```
$ cat savedforlater.json | jq -r '.[] | .title' | wc -l
    5915
$ 
```

Saved for Later には 5915 件の記事が登録されていた

## 2. 分かち書きする

```
$ cat savedforlater.json | jq -r '.[] | [.title, .abstract] | add' > savedforlater.txt # (1)
$ cat savedforlater.txt | mecab -Owakati > savedforlater-wakati.txt # (2)            
```

```
(1). JSON データ中のタイトル,概要部分だけ取り出す。jq の -r (raw output) オプションでダブルクオートで囲わずに出力させる
(2). 分かち書きにする。MeCab の辞書には mecab-ipadic-neologd を使用した
```

### 出力結果

```
$ less savedforlater-wakati.txt
```

```
Qiita レジェンド 達 の 偉大 さ を シェル芸 で 眺め て 2016年 を 振り返る 2016年 の 振り返り 私事 です が 、 2016年 の Qiita 振り返り を さ せ て いただき ます 。 2016年 に 投稿 し た Qiita の 記事 は 17 本 、 これ で 18 本 目 に なり ます 。 平均 する と 1 カ月 に 1 本 以上 書い て は いる の です が 、 時期 的 に は やはり 波 が あり ます ね 。 ネタ の ストック が なかっ た とか DevOps 導入 指南 執筆 の タイミング と 被っ た とかいう の も ある かも しれ ませ ん が 、 下期 は 全体 的 に 少なかっ た です 。 1 年 の 中 で 印象深かっ た の は 、 エンジニア なら 知っ て おき たい 、 絵 で 見 て わかる セキュア通信 の 基本 の 記事 で 、 ありがたい こと に これ だけ で 2000 を 超える いいね 1 を いただく こと が でき 、 他 の 記事 と あわせ て 年初 に 500 程度 だっ た 総 Contribution は 現時点 で 3700 を 超える まで に なり まし た 。 個人的 に こうした 数字 で 見える 成果 は 好き な ので 、 来年 も アウトプット を 続ける モチベーション に なっ た と 思っ て い ます 。 Cont 
シャイ ハック の 心得 を 説く ： 『 消極性 デザイン 宣言 』 出版記念 イベント レポート 消極性 を 再 定義 する 、 「 消極性 研究会 」 について おさらい 「 なぜ 、 自分 の ペース で コミュニケーション を 取ろ う と する と KY と 言わ れ て しまう ん だろ う 」 「 なぜ 、 この 社会 は やる気 が 無尽蔵 かつ 定常 的 に 得 られる と 無邪気 に 仮定 し て いる の だろ う か ？ 」 ▲ 津田塾大学 栗原 一貴 氏 当たり前 と さ れ て いる こうした こと に 疑問 を 感じ 、 情報科学 の 研究 者 たち が 立ち 上げ た の が 「 消極性 研究会 」 だ 。 メンバー は 2014年 の イグ ・ ノーベル賞受賞者 、 津田塾大学 の 栗原 一貴 氏 、 神戸大学 の 西田健志 氏 、 産業技術総合研究所 の 濱崎 雅弘 氏 、 Unity Technologies Japan ／ 慶應義塾大学 メディア デザイン 研究科 付属 メディア デザイン 研究所 リサーチャー の 簗 瀬 洋平 氏 、 『 融ける デザイン 』 の 明治大学 ・ 渡邊 恵太 氏 。 消極性 を コミュニケーション に対する 苦手 意識 という 意味 の シャイ と 、 日常 における やる気 の な さ や 「 ためらい 」 と とらえ 、 消極性 は 
2016年 はてな Web オペレーション エンジニア の アウトプット こんにちは 、 シニア エンジニア の id : y _ uuki です 。 はてな の Web オペレーション エンジニア 8 名 が 、 この 1 年 で 対外的 に アウトプット し た 成果物 を まとめ て 紹介 し ます 。 座談会 Software Engineering System Engineering 輪読 会 その他 あとがき 座談会 今週 、 Web オペレーション エンジニア の 座談会 記事 を 公開 し まし た 。 技術 ブログ や 技術 発表 で は 表現 し きれ ない 一 人 一 人 の 考え方 や 思い が 垣間見 える コンテンツ に なっ て いる と 思い ます 。 Software Engineering Keepalived の シンタックス チェッカ 「 gokc 」 を 作っ た - ゆ ううき ブログ gokc という Go で 作ら れ た Keepalived の 設定ファイル の シンタックス チェッカー です 。 社内 で 普通に 導入 し て い ます 。 インフラエンジニア が Go 言語 で オペレーション ツール を 書く こと について - Hatena 
メカデザイナー 大久保淳二 インタビュー 架空 企業 が 描く 未来 都市 の 構想 かつて 富野 由悠季 さん による ベストセラー 『 機動戦士ガンダム 』 を 再 構築 し た 小説 『 フォー・ザ・バレル ／ FOR THE BARREL 』 や 、 カルト 的 人気 を 誇る 操縦 ゲーム 『 鉄騎 』 の メカ デザイン を 担当 し た デザイナー の 大久保淳二 さん 。 近年 で は アニメ 『 魔法科高校の劣等生 』 や 『 オーバーロード 』 の プロップ デザイン など 、 TVアニメ で の 活躍 も 広く 知ら れ て いる 。 そんな 大久保淳二 さん が 、 「 近未来 の 都市部 に 働く ロボット 」 という コンセプト の もと 、 2000年 に スタート ... 
Redux の Action 、 Reducer 、 Store の （ 個人 的 な ） 整理 メモ はじめ に 今更 ながら です が 、 今 個人的 に 作っ て いる もの に React + Redux を 導入 する ついで に 、 いつ でも 確認 できる よう に 各 役割 について の メモ を 残し て おき ます 。 間違い など ござい まし たら 、 コメント にて ご 指摘 お待ち し て おり ます m(_ _)m ほぼ こちら に 英語 で 書い て ある こと に なり ます 。 Redux を 使っ て み た い方 は こちら の basics という 内容 を 追っ て いく と いい と 思い ます 。 http:// redux . js . org / docs / basics / Action ( http:// redux . js . org / docs / basics / Actions . html ) アクション は 『 何 か 』 が 起こっ た 時 、 Store に 『 どんな データ 』 を 利用 する か という こと を 定義 し ます 。 ActionCreator で ある store . dispatch () を 使っ て Store に 送信 し ます が 、 store について は この 記事 の 最後 に まとめ て い ます 。 
....
```

品詞ごとにスペース区切りのものが出来た

## 3. ベクタライズする

基本的には[こちら](http://kitashirakawakiriko.hatenablog.com/entry/2016/08/18/230702)を参考させてもらった

In [1]:
from gensim.models import doc2vec
import codecs

wakati_file = 'datasets/savedforlater-wakati.txt'
title_file = 'datasets/savedforlater.txt'

# 分かち書きデータ読み込み
wakati = doc2vec.TaggedLineDocument(wakati_file)

# 元のタイトルデータ読み込み（結果表示に使用する）
with codecs.open(title_file, 'r', encoding='utf-8') as f:    
    titles = [title.replace('\n', '') for title in f.readlines()]

In [2]:
# モデル作成
model = doc2vec.Doc2Vec(wakati)

In [3]:
# モデル保存
model.save('models/sfl.model')
model.save_word2vec_format('models/sfl.w2vmodel')

## 4. 似てる記事を推薦する

In [4]:
# 類似記事を見つけたい
test_title = u'Qiitaレジェンド達の偉大さをシェル芸で眺めて2016年を振り返る'
title_index = titles.index(test_title)

In [5]:
similar_docs = model.docvecs.most_similar(title_index)

print test_title
print "-" * 50
for similar_doc in similar_docs:
    # 類似度, インデックス, タイトル
    print ('%0.3f %4d %s') % (similar_doc[1], similar_doc[0], titles[similar_doc[0]])

Qiitaレジェンド達の偉大さをシェル芸で眺めて2016年を振り返る
--------------------------------------------------
0.545 4204 UI Tiles – Webサイト全体の画面遷移設計に使えるデザイン集
0.542   76 LT大会発表資料を公開！『ラズパイと友達に ～君の抵抗があったかいんだから編～』『Easy way to begin Computational Fluid Dynamics (CFD) on Cloud』
0.513 5491 【やじうまWatch】ドラえもんの「動くぬりえ」と同じ仕組みで動くフリーのぬりえアプリ
0.506 2265 Slackを置き換えるオープンソース・ソフトウェア5選 - THE BRIDGE（ザ・ブリッジ）
0.504 4927 ベイブレードをマンガのように操りたい①Arduino編
0.477  851 Giphy、GIFアニメを作成&編集できるMac用アプリ「GIPHY CAPTURE for Mac」v2をリリース。
0.477 2668 ヒロインに永遠に罵倒されるサイト
0.464 3684 お前なんかハッカーじゃない╭( ･ㅂ･)و ̑̑
0.461 3461 桜を見に行こう。ストリートビューで
0.459 1547 通勤や休憩時間に！スキマ時間を活用できる英語学習アプリ11選｜Sukimano


- タイトルだけじゃなくて、ある程度記事の内容も学習させないと微妙かも